In [135]:
from web3 import Web3, HTTPProvider
import json
from decimal import Decimal
import time 
import decimal
math = decimal.Context()
import numpy 
import random
%matplotlib inline
import matplotlib
import pandas

In [5]:
url = 'http://127.0.0.1:8545' 
w3 = Web3(Web3.HTTPProvider(url))

In [66]:
import web3

In [69]:
web3.types.Wei(10)

10

In [2]:
# Installing app
# npm init
# npm install --save-dev @nomiclabs/buidler
# npx buidler
# Follow instructions to create new project

# When project is ready
# Run npx buidler node

# run the script (Learn more: https://buidler.dev/guides/scripts.html )
# npx buidler --network localhost run scripts/pyTokenTest.js


# use web3 to advance blocks/block time with ganache:
# https://medium.com/fluidity/standing-the-time-of-test-b906fcc374a9

In [23]:
#Collateral address: 0x5bcb88A0d20426e451332eE6C4324b0e663c50E0
#Underlying address: 0x3521eF8AaB0323004A6dD8b03CE890F4Ea3A13f5
#Oracle address: 0x53369fd4680FfE3DfF39Fc6DDa9CfbfD43daeA2E
#pyToken address: 0xB00cC45B4a7d3e1FEE684cFc4417998A1c183e6d

In [24]:
collateral_contract_address = '0x5bcb88A0d20426e451332eE6C4324b0e663c50E0'
underlying_contract_address = '0x3521eF8AaB0323004A6dD8b03CE890F4Ea3A13f5'
oracle_contract_address = '0x53369fd4680FfE3DfF39Fc6DDa9CfbfD43daeA2E'
pyToken_contract_address = '0xB00cC45B4a7d3e1FEE684cFc4417998A1c183e6d'

In [25]:
# get the contract abi from the contract-project directory '../buidlertest/artifacts/Greeter.json'
with open('./dapp/artifacts/Collateral.json') as file:
    abi_json = json.load(file)
collateral_contract_abi = abi_json['abi']

# create the contract object
collateral = w3.eth.contract(address = Web3.toChecksumAddress(collateral_contract_address), 
                                          abi = collateral_contract_abi)

In [37]:
accounts = w3.eth.accounts

In [38]:
toMint = (100*10**18)
collateral.functions.mint(toMint).transact({'from': accounts[0]})

HexBytes('0x7f4888f829782b00924534c9ff2e3dbaabbf5b2c4a9c2f27e534174d53cebdb0')

In [39]:
collateral.functions.balanceOf(accounts[0]).call()

100000000000000000000

In [31]:
with open('./dapp/artifacts/Underlying.json') as file:
    abi_json = json.load(file)
underlying_contract_abi = abi_json['abi']

# create the contract object
underlying = w3.eth.contract(address = Web3.toChecksumAddress(underlying_contract_address), 
                                          abi = underlying_contract_abi)

In [40]:
toMint = (100*10**18)
underlying.functions.mint(toMint).transact({'from': accounts[0]})

HexBytes('0xe80039017e1041fe82c6c327144a9f86a6176b468f68cf46c8d6f0c1d159620b')

In [41]:
with open('./dapp/artifacts/Oracle.json') as file:
    abi_json = json.load(file)
oracle_contract_abi = abi_json['abi']

# create the contract object
oracle = w3.eth.contract(address = Web3.toChecksumAddress(oracle_contract_address), 
                                          abi = oracle_contract_abi)

In [43]:
price = (2*10**18)
oracle.functions.updatePrice(price).transact({'from': accounts[0]})

HexBytes('0x77512624332145d5b6630ecc0136dd60e08f0ffd7b36f0a1083fa0b79660e7a0')

In [46]:
with open('./dapp/artifacts/pyToken.json') as file:
    abi_json = json.load(file)
pytoken_contract_abi = abi_json['abi']

# create the contract object
pytoken = w3.eth.contract(address = Web3.toChecksumAddress(pyToken_contract_address), 
                                          abi = pytoken_contract_abi)

In [47]:
pytoken.functions.reservesTarget().call()

300000000000000000

## Block and time updates

In [92]:
w3.eth.blockNumber

38

In [115]:
def getBlockTimestamp():
    return w3.eth.getBlock(w3.eth.blockNumber).timestamp

def advanceBlock():
    w3.provider.make_request('evm_mine', [])
    
def advanceTime(seconds):
    w3.provider.make_request('evm_increaseTime', [seconds])
    
def takeSnapshot():
    return w3.provider.make_request('evm_snapshot', [])['result']

def revertToSnapShot(snapshot):
    return w3.provider.make_request('evm_revert', [snapshot])['result']

def getBlockNumber():
    return w3.eth.blockNumber

## Interacting with the contract

In [123]:
class Token:       
    def __init__(self, _name, _contract):
        self.balances = {}
        self.totalSupply = 0
        self._name = _name
        self.contract = _contract
    
    def mint(self, address, amount):
        self.contract.functions.mint(amount).transact({'from': address})
    
    def balanceOf(self, address):
        return self.contract.functions.balanceOf(address).call()
    
    def name(self):
        return self._name
    
    def transferFrom(self, sender, recipient, amount):
        assert(amount >= 0)
        return self.contract.functions.transfer(recipient, amount).transact({'from': sender})

In [124]:
token = Token("collateral", collateral)

In [127]:
token.mint(accounts[3], 100*10**18)

In [128]:
token.balanceOf(accounts[3])

300000000000000000000

In [130]:
token.transferFrom(accounts[3],accounts[4], int(1.7234 * 10**18))

HexBytes('0x7a45b3887ad770573c38962e5f5f6991619aefb9acdbafffd7fe40b4f4db66de')

In [131]:
token.balanceOf(accounts[4])

1723400000000000000

In [132]:
newAccount = w3.eth.accounts.create()

AttributeError: 'list' object has no attribute 'create'

# pyToken contract

In [ ]:
class pyToken ():
    def __init__(self, _collateral, _underlying, _pyToken):  
        self.underlying = _underlying
        self.collateral = _collateral
        self.pyToken = _pyToken
    
    #View functions
    def yearlyDebtRate(self):
        return math.power(self.debtRate, Decimal(31622400)) - 1
    
    def getTotalDebt(self):
        pass
    
    def getCollateral(self, token, address):
        pass
    
    def getDebt(self, token, address):
        pass
    
    def debtInUnderlying(self, token, address):
        pass
    
    def balanceOfUnderlying(self, address):
        pass
    
    def yearlyRate(self):
        rate = self.pytoken.functions.lastBlockInterest().call()
        return math.power(rate, Decimal(31622400)) - 1
    
    def totalSupplyUnderlying(self):
        pass
    
    def underlyingHeld(self):
        return self.pytoken.functions.underlyingHeld().call()
    
    
    #ERC-20
    def transferUnderlying(self, sender, recipient, amount):
        value = amount / self.rateAccumulator
        return self.transferFrom(sender, recipient, value)
    
    # User Functions
    
    def mint(self, address, amount):
        return pytoken.functions.mint(amount).transact({'from': address})
    
    def redeem(self, address, amount):
        return pytoken.functions.redeem(amount).transact({'from': address})
        
    def mintInUnderlying(self, address, amount):
        pytoken.functions.mint(amount).transact({'from': address})
        return self.mint(address, amount/self.rateAccumulator)
        
    def burnInUnderlying(self, address, amount):
        return self.burn(address, amount/self.rateAccumulator)
    
    def addCollateral(self, token, address, amount):
        return pytoken.functions.addCollateral(address, amount).transact({'from': address})
      
    def withdrawCollateral(self, token, address, amount):
        return pytoken.functions.withdrawCollateral(address, amount).transact({'from': address})
    
    # accrue for both debt and interest
    def accrueInterest(self):
        return pytoken.functions.accrueInterest().transact()
    
    def updateRates(self):
        return pytoken.functions.updateRates().transact()
    
    # borrow against a particular tokens worth of collateral
    def borrow(self, token, address, amount):
         pass
        
    def maximumBorrow(self, token, address):
        pass

       
    def liquidate(self, token, addressToLiquidate, LiquidatorAddress, amount):
        pass
        
    # Position in a particular collateral token
    def position(self, token, address):
        pass
    
    def repay(self, token, address, amount):
        pass


## Simulation

In [ ]:
class blockchain:
    
    def __init__(self, _timestamp, _block):
        self.timestamp = _timestamp
        self.block = _block
    
    def incrementBlock(self):
        advanceBlock()
    
    def updateTimestamp(self, _timestamp):
        seconds = _timestamp - getBlockTimestamp()
        advanceTime(seconds)
        advanceBlock()
    
    def blockNumber():
        return getBlockNumber()
    
    def now(self):
        return getBlockTimestamp()

In [113]:
class Interface:
    
    def __init__(self, _chain : blockchain, _uTokenA : Token, _uTokenB : Token, _oracle, _pyA):
        self.A = _uTokenA
        self.B = _uTokenB
        self.chain = _chain
        self.oracle = _oracle
        self.pyA = _pyA
        
    def updateInterestRates(self):
        self.pyA.updateRates()
        self.pyA.accrueInterest()
    
    def eachBlock(self):
        self.chain.incrementBlock()
        self.updateInterestRates()
        


True

In [ ]:
class Agent:
    def __init__(self, _interface):
        self.interface = _interface
        self.address = numpy.random.randint(1,1000)
        
    def setup(self):
        pass
    
    def performActions(self):
        pass
    
    def shutdown(self):
        pass

In [ ]:
class YieldAgent(Agent):
    def __init__(self, _interface):
        super().__init__(_interface)
        self.address = "0xYA_HodlPY" + str(self.address)
        self.block = 0
        
    def setup(self):   
        self.initialA = min(Decimal(round(numpy.random.lognormal(6, 1))), Decimal(1000))
        self.interface.A.mint(self.address, Decimal(self.initialA))
        self.lowerRate = Decimal(numpy.random.uniform(0,7) / 100)
        self.upperRate = self.lowerRate + Decimal(numpy.random.uniform(0,5) / 100)
    
    def performActions(self):
        if self.interface.pyA.yearlyRate() > self.upperRate:
            self.HODL()
        elif self.interface.pyA.yearlyRate() < self.lowerRate:
            self.unHODL()
            
    def HODL(self):
        self.block = self.interface.chain.blockNumber()
        if self.interface.A.balanceOf(self.address) > 0:
            aBalance = self.interface.A.balanceOf(self.address)
            self.interface.pyA.mint(self.address, aBalance)
            print("HODLing {} tokens at rate {}".format(aBalance, round(self.upperRate, 4)))
            
    def unHODL(self):
        if self.block - self.interface.chain.blockNumber() < 10:
            return
        amount = min(self.interface.pyA.balanceOfUnderlying(self.address), self.interface.pyA.underlyingHeld())
        amount = amount*Decimal('0.1')
        if amount > 0:
            # let's only remove 10% at a time
            self.interface.pyA.redeem(self.address, amount)
            print("unHODLing {} tokens".format(amount, round(self.lowerRate, 4)))
            
            

In [114]:

    


# Borrow agents lever up on ETH collateral by selling pyTokens
class BorrowAgent(Agent):
    def __init__(self, _interface):
        super().__init__(_interface)
        self.address = "0xBA_longETH" + str(self.address)
        
    def setup(self):
        self.initialETH = min(Decimal(round(numpy.random.lognormal(3, 1))), Decimal(100))
        self.interface.eth.mint(self.address, Decimal(self.initialETH ))
        ethReserve = self.initialETH * Decimal('0.1')
        self.each = (self.initialETH - ethReserve)/2
        self.interface.pyA.addCollateral("ETH", self.address, self.each )
        self.interface.pyB.addCollateral("ETH", self.address, self.each )
        price = self.interface.ethAOracle.getPriceInToken(self.interface.A)
        self.desiredBorrow = Decimal('0.5')*self.interface.pyA.maximumBorrow("ETH", self.address)
        self.interface.A.mint(self.address, self.desiredBorrow*1000)
        self.lowerRate = Decimal(numpy.random.uniform(0,10) / 100)
        self.upperRate = self.lowerRate + Decimal(numpy.random.uniform(0,5) / 100)
    
    def performActions(self):
        if self.interface.pyA.yearlyDebtRate() > self.upperRate:
            self.unBorrow()
        elif self.interface.pyA.yearlyDebtRate() < self.lowerRate:
            self.borrow()
        uAmount = min(self.interface.pyA.balanceOfUnderlying(self.address), self.interface.qxA.underlying)
        if uAmount > Decimal('0.1'):
            self.interface.qxA.redeem(self.address, uAmount*Decimal(.99))
            print("Withdrew {} tokens at {}".format(round(uAmount*Decimal(.99), 4), round(self.lowerRate, 4) ))
            
            
    def borrow(self):
        price = self.interface.ethAOracle.getPriceInToken(self.interface.A)
        #if we've already mostly borrowed what we desire, let's stop
        if self.interface.pyA.debtInUnderlying("ETH", self.address) > Decimal('.99') * self.desiredBorrow:
            return
        #initiate borrow
        borrowRatio = Decimal('0.5')
        amount = self.interface.pyA.maximumBorrow("ETH", self.address) - self.interface.pyA.debtInUnderlying("ETH", self.address)
        borrow = (amount * borrowRatio) / (self.interface.pyA.collateralizationRatio )
        if borrow > Decimal('0.1'):
            self.interface.pyA.borrow("ETH", self.address, borrow)
            print("Borrowed {} tokens at {}".format(round(borrow, 4), round(self.lowerRate, 4) ))

    
    def unBorrow(self):
        amount = Decimal('0.95') * self.interface.pyA.debtInUnderlying("ETH", self.address)
        #funds = self.interface.A.balanceOf(self.address)
        #toUnBorrow = min(debt, funds)
        if amount > Decimal('0.1'):
            try:
                self.interface.qxA.mint(self.address, amount*Decimal(1.01))
                self.interface.pyA.repay("ETH", self.address, amount)
                print("unBorrowed {} tokens at {}".format(round(amount, 4), round(self.upperRate, 4) ))
            except Exception as e: 
                print("failed to unborrow. amount:{} underlying balance: {} pyTokens: {} ".format(amount, self.interface.A.balanceOf(self.address), self.interface.pyA.balanceOfUnderlying(self.address)))
                raise e
    


38